In [ ]:
vals = [5, 4, 6]
weights = [2, 3, 4]
cap = 5
# CREATING array of possible penalty values
penalties = np.linspace(3.5, 5.5, 10)



#CREATING array to track the best values across all the penalties tested
results = []
best_run = (None, None, -float("inf"))

# problem hamiltonian
# TRANSALTING ham prob to parse through array
hammies = []
for pens in penalties:
    hams = create_knapsack_hamiltonian(weights, vals, cap, pens) #create the hamils
    hammies.append(hams) #store hamils because we paranoid 

    ansatz1 = QAOAAnsatz(hams, reps=2) #run the ansatz coz jai did 
#QUESTION
    #estimator = StatevectorEstimator() # does this need to be loop?
    x0 = 2 * np.pi * np.random.rand(ansatz1.num_parameters) # best to keep this loop not sure if it changes(?)
    # cost = cost_func(x0, ansatz, hamiltonian, estimator) #for debug 
    res1 = minimize(cost_func, x0, 
               args=(ansatz1, hams, estimator),
               method="COBYLA") # assuming this solves the hamil w/cobyla

    qc1 = ansatz1.decompose(reps=3)
    qc1 = qc1.copy()
    qc1.measure_all()
    bound = qc1.assign_parameters(res1.x)

    backend = AerSimulator()
    tqc = transpile(bound, backend)
    job1 = backend.run(tqc, shots=1024)
    counts1 = job1.result().get_counts()

    total1 = sum(counts1.values())
    dist1 = {int(bitstr, 2): cnt / total1 for bitstr, cnt in counts1.items()}

    n1 = len(vals)

    best1 = (None, -1, None, 0.0)  # (x_bits, value, weight, prob)
    for k, p in dist1.items():
        x = bits_from_int(k, n1)
        val, wt = score(x)
        if wt <= cap and val > best1[1]: # check best within penalty value
            best1 = (x, val, wt, float(p))
            results.append((pens, *best1, res1))
        if best1[1] > best_run[2]: # check best across penalty values
            best_run = (pens, best1, best1[1])

pens_best, (x_bits_best, best_val, best_wt, best_prob), _ = best_run
picked1 = [i for i, b in enumerate(x_bits_best or []) if b]

print("\nBest (feasible) sample across all penalties:")
print("  Penalty      :", pens_best)
print("  items picked:", picked1)
print("  bitstring   :", ''.join(map(str, x_bits_best or [])))
print("  value/weight:", best_val, "/", best_wt, f"(capacity {cap})")
print("  sample prob :", best_prob)
os.system('say "WHOOO HOOO!"')
    